In [1]:
spark

Waiting for a Spark session to start...

org.apache.spark.sql.SparkSession@3695dfc0

In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.{col}

In [4]:
val spark = SparkSession.builder.appName("korneev").getOrCreate()

spark = org.apache.spark.sql.SparkSession@3695dfc0


org.apache.spark.sql.SparkSession@3695dfc0

In [5]:
var train = spark
            .read
            .option("header", true)
            .option("inferSchema", true)
            .csv("/labs/slaba05/lab05_trbain.csv")

train.show(1, 0, true)

-RECORD 0--------------------------------------------
 _c0                         | 333149                
 ID                          | 479990                
 CR_PROD_CNT_IL              | 0                     
 AMOUNT_RUB_CLO_PRC          | 0.0                   
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                   
 APP_REGISTR_RGN_CODE        | null                  
 PRC_ACCEPTS_A_POS           | 0.0                   
 PRC_ACCEPTS_A_TK            | 0.0                   
 TURNOVER_DYNAMIC_IL_1M      | 0.0                   
 CNT_TRAN_AUT_TENDENCY1M     | null                  
 SUM_TRAN_AUT_TENDENCY1M     | null                  
 AMOUNT_RUB_SUP_PRC          | 0.0                   
 PRC_ACCEPTS_A_AMOBILE       | 0.0                   
 SUM_TRAN_AUT_TENDENCY3M     | null                  
 CLNT_TRUST_RELATION         | null                  
 PRC_ACCEPTS_TK              | 0.0                   
 PRC_ACCEPTS_A_MTP           | 0.0                   
 REST_DYNAMIC_FDEP_1M       

train = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [6]:
//Пакет услуг
train.groupBy("PACK").agg(count("ID")).show(5)

+----+---------+
|PACK|count(ID)|
+----+---------+
| 101|     1645|
| 107|    25108|
|null|        1|
| K01|    69478|
| M01|        5|
+----+---------+
only showing top 5 rows



In [7]:
//Тип компании (основной работы клиента)
train.groupBy("APP_COMP_TYPE").agg(count("ID")).show(5)

+-------------+---------+
|APP_COMP_TYPE|count(ID)|
+-------------+---------+
|        INTER|     1206|
|         null|   259778|
|        STATE|     5451|
|           IP|      800|
|      PRIVATE|    53529|
+-------------+---------+



In [8]:
//Тип работы (занятость)
train.groupBy("APP_EMP_TYPE").agg(count("ID")).show(5)

+------------+---------+
|APP_EMP_TYPE|count(ID)|
+------------+---------+
|       INTER|     1206|
|        null|   259778|
|       STATE|     5451|
|          IP|      800|
|     PRIVATE|    53529|
+------------+---------+



In [9]:
train.groupBy("CLNT_TRUST_RELATION").agg(count("ID")).show(30)

+-------------------+---------+
|CLNT_TRUST_RELATION|count(ID)|
+-------------------+---------+
|               Мать|       80|
|             FRIEND|    22613|
|             SISTER|     5017|
|               мать|        1|
|                Сын|        7|
|               null|   257829|
|           DAUGHTER|     1837|
|             MOTHER|    10046|
|            BROTHER|     5080|
|               Жена|        3|
|             FATHER|     2744|
|         Близкий ро|       39|
|         Дальний ро|        7|
|                Муж|        3|
|             Сестра|       15|
|               Отец|       26|
|              OTHER|     6048|
|               Брат|       26|
|               Дочь|        8|
|                SON|     1715|
|               Друг|      188|
|           RELATIVE|     7431|
|               мама|        1|
+-------------------+---------+



In [10]:
train.groupBy("APP_MARITAL_STATUS").agg(count("ID")).show(5)
train.groupBy("APP_KIND_OF_PROP_HABITATION").agg(count("ID")).show(5)
train.groupBy("CLNT_JOB_POSITION_TYPE").agg(count("ID")).show(5)
train.groupBy("CLNT_JOB_POSITION").agg(count("ID")).show(5)
train.groupBy("APP_DRIVING_LICENSE").agg(count("ID")).show(5)
train.groupBy("APP_EDUCATION").agg(count("ID")).show(5)
train.groupBy("APP_TRAVEL_PASS").agg(count("ID")).show(5)
train.groupBy("APP_CAR").agg(count("ID")).show(5)
train.groupBy("APP_POSITION_TYPE").agg(count("ID")).show(5)
train.groupBy("APP_EMP_TYPE").agg(count("ID")).show(5)
train.groupBy("APP_COMP_TYPE").agg(count("ID")).show(5)
train.groupBy("PACK").agg(count("ID")).show(5)

+------------------+---------+
|APP_MARITAL_STATUS|count(ID)|
+------------------+---------+
|                 m|     1298|
|              null|   258989|
|                 T|     2932|
|                 M|    27848|
|                 V|    20116|
+------------------+---------+
only showing top 5 rows

+---------------------------+---------+
|APP_KIND_OF_PROP_HABITATION|count(ID)|
+---------------------------+---------+
|                    NPRIVAT|     2328|
|                       null|   267042|
|                         SO|    25461|
|                       RENT|     2931|
|                         JO|    12750|
+---------------------------+---------+
only showing top 5 rows

+----------------------+---------+
|CLNT_JOB_POSITION_TYPE|count(ID)|
+----------------------+---------+
|                  null|   280242|
|            SPECIALIST|    22741|
|           TOP_MANAGER|     6605|
|               MANAGER|     8464|
|             SELF_EMPL|     2712|
+----------------------+-------

In [11]:
// model 
import org.apache.spark.ml.feature.OneHotEncoder
import org.apache.spark.ml.feature.{VectorAssembler}


In [12]:
val dropColumns = List(
    "CLNT_TRUST_RELATION", 
    "APP_MARITAL_STATUS",
    "APP_KIND_OF_PROP_HABITATION", 
    "CLNT_JOB_POSITION_TYPE",
    "CLNT_JOB_POSITION", 
    "APP_DRIVING_LICENSE", 
    "APP_EDUCATION",
    "APP_TRAVEL_PASS", 
    "APP_CAR", 
    "APP_POSITION_TYPE", 
    "APP_EMP_TYPE",
    "APP_COMP_TYPE", 
    "PACK"
)
train = train.drop(dropColumns:_*).na.fill(0)

dropColumns = List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)
train = [_c0: int, ID: int ... 102 more fields]


[_c0: int, ID: int ... 102 more fields]

In [13]:
val vectorAssembler = new VectorAssembler()
    .setInputCols(train.drop("TARGET").columns)
    .setOutputCol("features")

vectorAssembler = vecAssembler_2ea8c289d853


vecAssembler_2ea8c289d853

In [14]:
val trainData = vectorAssembler.transform(train).cache

trainData = [_c0: int, ID: int ... 103 more fields]


[_c0: int, ID: int ... 103 more fields]

In [15]:
// Split the data into training and test sets (30% held out for testing).
val Array(trainingData, validData) = trainData.randomSplit(Array(0.8, 0.2))

trainingData = [_c0: int, ID: int ... 103 more fields]
validData = [_c0: int, ID: int ... 103 more fields]


[_c0: int, ID: int ... 103 more fields]

In [16]:
import org.apache.spark.ml.classification.{RandomForestClassifier, GBTClassifier}

In [18]:
// Train GBT
val clf = new GBTClassifier()
    .setLabelCol("TARGET")
    .setFeaturesCol("features")
    .setMaxIter(10)
    .setFeatureSubsetStrategy("auto")
    .fit(trainingData)

clf = GBTClassificationModel (uid=gbtc_fdd4aa3d74d4) with 10 trees


GBTClassificationModel (uid=gbtc_fdd4aa3d74d4) with 10 trees

In [19]:
val predictions = clf.transform(validData).cache

predictions = [_c0: int, ID: int ... 106 more fields]


[_c0: int, ID: int ... 106 more fields]

In [20]:
predictions.select(
        "TARGET",
        "features",
        "rawPrediction",
        "probability",
        "prediction"
).show(5)

+------+--------------------+--------------------+--------------------+----------+
|TARGET|            features|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+--------------------+----------+
|     0|(103,[1,19,21,35,...|[1.08377333464616...|[0.89729710009515...|       0.0|
|     0|(103,[0,1,3,9,10,...|[1.04269642120368...|[0.88947531272972...|       0.0|
|     0|(103,[0,1,3,9,10,...|[1.03025024197624...|[0.88700434196973...|       0.0|
|     0|(103,[0,1,23,29,3...|[1.11800605892970...|[0.90343712075233...|       0.0|
|     0|(103,[0,1,5,11,23...|[1.01033999769305...|[0.88295130379303...|       0.0|
+------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [21]:
predictions.select("probability").show(1, 0, true)

-RECORD 0----------------------------------------------
 probability | [0.8972971000951554,0.1027028999048446] 
only showing top 1 row



In [22]:
// import org.apache.spark.ml.linalg.Vector
// val vectorSecondElementUdf = udf((v: Vector) => v.toArray(1))
// требуется DoublyType 
// лень возиться
// нагуглил UDF vectorToArrayUdf
//
//
// https://stackoverflow.com/questions/46053318/convert-a-spark-vector-of-features-into-an-array

import org.apache.spark.ml.linalg.{SparseVector, Vector}
import org.apache.spark.mllib.linalg.{Vector => OldVector}

val vectorToArrayUdf = udf { vec: Any =>
    vec match {
      case v: Vector => v.toArray
      case v: OldVector => v.toArray
      case v => throw new IllegalArgumentException(
        "function vector_to_array requires a non-null input argument and input type must be " +
        "`org.apache.spark.ml.linalg.Vector` or `org.apache.spark.mllib.linalg.Vector`, " +
        s"but got ${ if (v == null) "null" else v.getClass.getName }.")
    }
  }.asNonNullable()

vectorToArrayUdf = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)


import org.apache.spark.mllib.linalg.{Vector=>OldVector}


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)

In [23]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

In [24]:
val evaluator = new BinaryClassificationEvaluator()
    .setLabelCol("TARGET")
    .setRawPredictionCol("prediction")
    .setMetricName("areaUnderROC")

val accuracy = evaluator
    .evaluate(
        predictions.withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    )

evaluator = binEval_fc2f6a5b9bfe
accuracy = 0.8176650061622104


0.8176650061622104

In [25]:
var test = spark
            .read
            .option("header", true)
            .option("inferSchema", true)
            .csv("/labs/slaba05/lab05_test.csv")
            .na.fill(0)
// test.show(1)

test = [_c0: int, ID: int ... 114 more fields]


[_c0: int, ID: int ... 114 more fields]

In [26]:
val pred_test = clf.transform(vectorAssembler.transform(test)).cache

pred_test = [_c0: int, ID: int ... 118 more fields]


[_c0: int, ID: int ... 118 more fields]

In [27]:
pred_test.select("ID", "probability")
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .show(5)

+------+--------------------+-------------------+
|    ID|         probability|         prediction|
+------+--------------------+-------------------+
|519130|[0.83060102665691...| 0.1693989733430803|
|234045|[0.92713343258804...|0.07286656741195363|
|401256|[0.92952619429671...|0.07047380570328976|
|551070|[0.90401660513344...|0.09598339486655216|
|367285|[0.92005838308264...|0.07994161691735746|
+------+--------------------+-------------------+
only showing top 5 rows



In [30]:
pred_test.withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select(col("ID").alias("id"), col("prediction").alias("target"))
    .coalesce(1)
    .write
    .format("csv")
    .mode("overwrite")
    .option("header", "true")
    .option("sep", "\t")
    .save("lab05.csv")

In [41]:
pred_test.withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select(col("ID").alias("id"), col("prediction").alias("target"))
    .coalesce(1)
    .write
    .format("csv")
    .mode("overwrite")
    .option("header", "true")
    .option("sep", "\t")
    .save("lab05.csv") 

lastException: Throwable = null


Name: Compile Error
Message: <console>:37: error: not found: value hdfs
       ! hdfs dfs -get /user/andrey.korneev/lab05.csv
         ^
<console>:37: error: not found: value get
       ! hdfs dfs -get /user/andrey.korneev/lab05.csv
                   ^
<console>:37: error: not found: value user
       ! hdfs dfs -get /user/andrey.korneev/lab05.csv
                        ^
<console>:37: error: not found: value andrey
       ! hdfs dfs -get /user/andrey.korneev/lab05.csv
                             ^
<console>:37: error: not found: value lab05
       ! hdfs dfs -get /user/andrey.korneev/lab05.csv
                                            ^

StackTrace: 